# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [0]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm, trange
from torch import optim
from torch.autograd import Variable
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [0]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [0]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.layer_1 = torch.nn.Sequential(torch.nn.Conv2d(1, 128, kernel_size = 3, stride = 1, padding = 1),
                                        torch.nn.ReLU(),
                                        torch.nn.Conv2d(128, 512, kernel_size = 3, stride = 1, padding = 1),
                                        torch.nn.ReLU(),
                                        torch.nn.MaxPool2d(stride = 2, kernel_size = 2))
        self.layer_2 = torch.nn.Sequential(torch.nn.Linear(14 * 14  * 512, 2048),
                                        torch.nn.ReLU(),
                                        torch.nn.Dropout(p = 0.5),
                                        torch.nn.Linear(2048, 10))
    
    def forward(self, x):
        x = self.layer_1(x)
        x = x.view(-1, 14 * 14 * 512)
        x = self.layer_2(x)
        return x
    
model = SimpleNet()
model.cuda()

# TODO:define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0002)

Next, we can start to train and evaluate!

In [16]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    train_correct, train_total = 0, len(train_dataset)
    test_correct, test_total = 0, len(test_dataset)
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        X_train, y_train = Variable(images).cuda(), Variable(labels).cuda()
        optimizer.zero_grad()
        outputs = model(X_train)
        _, y_pred = torch.max(outputs, 1)
        train_correct += torch.sum(y_pred == y_train.data)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    for images, labels in tqdm(test_loader):
        X_test, y_test = Variable(images).cuda(), Variable(labels).cuda()
        outputs = model(X_test)
        _, y_pred = torch.max(outputs, 1)
        test_correct += torch.sum(y_pred == y_test.data)
    
    print("Training Accuracy: %.2f%%" % (train_correct.cpu().numpy() / train_total * 100))
    print("Testing Accuracy: %.2f%%" % (test_correct.cpu().numpy() / test_total * 100))

  0%|          | 2/468 [00:00<00:28, 16.18it/s]

Training Accuracy: 94.20%
Testing Accuracy: 97.98%


  0%|          | 2/468 [00:00<00:28, 16.63it/s]

Training Accuracy: 98.50%
Testing Accuracy: 98.68%


  0%|          | 2/468 [00:00<00:28, 16.40it/s]

...

Training Accuracy: 99.57%
Testing Accuracy: 98.75%


  0%|          | 2/468 [00:00<00:27, 16.95it/s]

Training Accuracy: 99.64%
Testing Accuracy: 98.59%


100%|██████████| 78/78 [00:01<00:00, 41.47it/s]

Training Accuracy: 99.62%
Testing Accuracy: 98.90%


#### Q5:
Please print the training and testing accuracy.

In [17]:
model.eval()
train_correct, train_total = 0, len(train_dataset)
test_correct, test_total = 0, len(test_dataset)

for images, labels in tqdm(train_loader):
    X_train, y_train = Variable(images).cuda(), Variable(labels).cuda()
    outputs = model(X_train)
    _, y_pred = torch.max(outputs, 1)
    train_correct += torch.sum(y_pred == y_train.data)

for images, labels in tqdm(test_loader):
    X_test, y_test = Variable(images).cuda(), Variable(labels).cuda()
    outputs = model(X_test)
    _, y_pred = torch.max(outputs, 1)
    test_correct += torch.sum(y_pred == y_test.data)
    
print("Training Accuracy: %.2f%%" % (train_correct.cpu().numpy() / train_total * 100))
print("Testing Accuracy: %.2f%%" % (test_correct.cpu().numpy() / test_total * 100))

100%|██████████| 78/78 [00:01<00:00, 43.37it/s]


Training Accuracy: 99.81%
Testing Accuracy: 99.06%
